<a href="https://colab.research.google.com/github/yakshteja2004/Oasis-Infobyte-/blob/main/Autocomplete_and_Autocorrect_Data_Analytics_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Run this cell in Colab to install libraries that might not be preinstalled
!pip install plotly==5.15.0 textblob==0.17.1 nltk==3.8.1 --quiet

# For TextBlob corpora
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 107.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 636.8/636.8 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 39.1 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
# Common imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

from textblob import TextBlob
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split

# Display options
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 100)
sns.set(style="whitegrid")

In [5]:
# Replace the path/url below with your dataset link or upload the file to Colab and set the correct path.
# Example common filenames: 'googleplaystore.csv', 'apps.csv', 'playstore.csv'
DATA_PATH = "/content/apps.csv"  # <- replace with your Dataset 1 Link or uploaded filename

# If you have a URL and it's raw CSV on the web, you can use:
# DATA_PATH = "https://path/to/your/googleplaystore.csv"

df = pd.read_csv(DATA_PATH)
df.head()


,Unnamed: 0,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19.0,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,1,Coloring book moana,ART_AND_DESIGN,3.9,967,14.0,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25.0,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [6]:
# Quick info
print("Rows, Columns:", df.shape)
display(df.info())
display(df.describe(include='all').T)

# Missing values summary
missing = df.isnull().sum().sort_values(ascending=False)
display(missing[missing > 0])


Rows, Columns: (9659, 14)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9659 entries, 0 to 9658
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      9659 non-null   int64  
 1   App             9659 non-null   object 
 2   Category        9659 non-null   object 
 3   Rating          8196 non-null   float64
 4   Reviews         9659 non-null   int64  
 5   Size            8432 non-null   float64
 6   Installs        9659 non-null   object 
 7   Type            9659 non-null   object 
 8   Price           9659 non-null   object 
 9   Content Rating  9659 non-null   object 
 10  Genres          9659 non-null   object 
 11  Last Updated    9659 non-null   object 
 12  Current Ver     9651 non-null   object 
 13  Android Ver     9657 non-null   object 
dtypes: float64(2), int64(2), object(10)
memory usage: 1.0+ MB


None

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Unnamed: 0,9659.0,NaN,NaN,NaN,5666.172896,3102.362863,0.0,3111.5,5814.0,8327.5,10840.0
App,9659,9659,iHoroscope - 2018 Daily Horoscope & Astrology,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Category,9659,33,FAMILY,1832,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Rating,8196.0,NaN,NaN,NaN,4.173243,0.536625,1.0,4.0,4.3,4.5,5.0
Reviews,9659.0,NaN,NaN,NaN,216592.563826,1831319.566176,0.0,25.0,967.0,29401.0,78158306.0
Size,8432.0,NaN,NaN,NaN,20.395327,21.827509,0.0,4.6,12.0,28.0,100.0
Installs,9659,21,"1,000,000+",1417,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Type,9659,2,Free,8903,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Price,9659,92,0,8903,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Content Rating,9659,6,Everyone,7903,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,0
Rating,1463
Size,1227
Current Ver,8
Android Ver,2


In [7]:
# Make a copy
data = df.copy()

# Common Play Store dataset cleaning steps (adapt to your dataset columns)

# 1) Drop duplicates if any
data.drop_duplicates(inplace=True)

# 2) Normalize column names
data.columns = [c.strip().lower().replace(' ', '_') for c in data.columns]

# 3) Convert rating to numeric
if 'rating' in data.columns:
    data['rating'] = pd.to_numeric(data['rating'], errors='coerce')

# 4) Clean installs (e.g. "1,000,000+" -> 1000000)
if 'installs' in data.columns:
    data['installs'] = data['installs'].astype(str).str.replace('+','').str.replace(',','')
    data['installs'] = pd.to_numeric(data['installs'], errors='coerce')

# 5) Clean size (e.g. '19M', 'Varies with device')
def parse_size(x):
    try:
        x = x.strip()
        if x.lower().endswith('m'):
            return float(x[:-1]) * 1e6
        if x.lower().endswith('k'):
            return float(x[:-1]) * 1e3
        if x.lower() == 'varies with device':
            return np.nan
        return float(x)
    except:
        return np.nan

if 'size' in data.columns:
    data['size_bytes'] = data['size'].astype(str).apply(parse_size)

# 6) Convert price (e.g. '$2.99' -> 2.99)
if 'price' in data.columns:
    data['price'] = data['price'].astype(str).str.replace('$','').str.strip()
    data['price'] = pd.to_numeric(data['price'], errors='coerce')

# 7) Convert last_updated to datetime if present
if 'last_updated' in data.columns:
    data['last_updated'] = pd.to_datetime(data['last_updated'], errors='coerce')

# 8) Fill or drop NA as strategy — for now we'll keep as-is and highlight rows with important missing values
print("After cleaning: Rows, Columns:", data.shape)
display(data.head())

After cleaning: Rows, Columns: (9659, 15)


,unnamed:_0,app,category,rating,reviews,size,installs,type,price,content_rating,genres,last_updated,current_ver,android_ver,size_bytes
0,0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19.0,10000,Free,0.0,Everyone,Art & Design,2018-01-07,1.0.0,4.0.3 and up,19.0
1,1,Coloring book moana,ART_AND_DESIGN,3.9,967,14.0,500000,Free,0.0,Everyone,Art & Design;Pretend Play,2018-01-15,2.0.0,4.0.3 and up,14.0
2,2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7,5000000,Free,0.0,Everyone,Art & Design,2018-08-01,1.2.4,4.0.3 and up,8.7
3,3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25.0,50000000,Free,0.0,Teen,Art & Design,2018-06-08,Varies with device,4.2 and up,25.0
4,4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8,100000,Free,0.0,Everyone,Art & Design;Creativity,2018-06-20,1.1,4.4 and up,2.8


In [8]:
# Top categories by app count
if 'category' in data.columns:
    cat_count = data['category'].value_counts().reset_index()
    cat_count.columns = ['category', 'count']
    display(cat_count.head(20))

    # Bar chart (top 15 categories)
    fig = px.bar(cat_count.head(15), x='category', y='count',
                 title='Top 15 Categories by Number of Apps', labels={'count':'# apps'})
    fig.update_layout(xaxis_tickangle=-45)
    fig.show()

,category,count
0,FAMILY,1832
1,GAME,959
2,TOOLS,827
3,BUSINESS,420
4,MEDICAL,395
5,PERSONALIZATION,376
6,PRODUCTIVITY,374
7,LIFESTYLE,369
8,FINANCE,345
9,SPORTS,325


In [9]:
# Rating distribution
if 'rating' in data.columns:
    fig = px.histogram(data, x='rating', nbins=20, title='Distribution of App Ratings')
    fig.show()

# Installs distribution (log scale)
if 'installs' in data.columns:
    data['installs_log'] = np.log1p(data['installs'])
    fig = px.histogram(data, x='installs_log', nbins=50, title='Log Distribution of Installs')
    fig.show()

# Price vs Rating (box)
if {'price','rating'}.issubset(data.columns):
    fig = px.box(data[data['price'].notna() & (data['price']>0)], x='price', y='rating',
                 title='Rating Distribution for Paid Apps (Price vs Rating)', points='outliers')
    fig.update_xaxes(type='category')
    fig.show()

# Avg rating per category
if {'category','rating'}.issubset(data.columns):
    avg_rating = data.groupby('category')['rating'].mean().reset_index().sort_values(by='rating', ascending=False)
    display(avg_rating.head(15))
    fig = px.bar(avg_rating.head(15), x='category', y='rating', title='Top 15 Categories by Average Rating')
    fig.update_layout(xaxis_tickangle=-45)
    fig.show()

,category,rating
10,EVENTS,4.435556
8,EDUCATION,4.364407
0,ART_AND_DESIGN,4.357377
3,BOOKS_AND_REFERENCE,4.344970
23,PERSONALIZATION,4.332215
22,PARENTING,4.300000
2,BEAUTY,4.278571
14,GAME,4.247368
27,SOCIAL,4.247291
32,WEATHER,4.243056


In [10]:
if {'app','installs'}.issubset(data.columns):
    top_installs = data.sort_values('installs', ascending=False)[['app','category','installs','rating']].head(20)
    display(top_installs)

    fig = px.scatter(top_installs, x='app', y='installs', size='installs', color='rating',
                     hover_data=['category','rating'], title='Top 20 Apps by Installs')
    fig.update_layout(xaxis_tickangle=-45)
    fig.show()

,app,category,installs,rating
2977,Google News,NEWS_AND_MAGAZINES,1000000000,3.9
2012,Google+,SOCIAL,1000000000,4.2
1356,Subway Surfers,GAME,1000000000,4.5
152,Google Play Books,BOOKS_AND_REFERENCE,1000000000,3.9
2908,YouTube,VIDEO_PLAYERS,1000000000,4.3
2003,Instagram,SOCIAL,1000000000,4.5
2930,Google Play Movies & TV,VIDEO_PLAYERS,1000000000,3.7
2191,Google Photos,PHOTOGRAPHY,1000000000,4.5
2429,Google Street View,TRAVEL_AND_LOCAL,1000000000,4.2
701,Google Play Games,ENTERTAINMENT,1000000000,4.3


In [11]:
# This is a simple polarity-based sentiment analysis using TextBlob.
# If your dataset doesn't include reviews, skip this cell or load a reviews dataset.
if 'reviews' in data.columns:
    # If reviews is numeric (count), you may have a separate reviews dataset. Here we assume text reviews exist.
    # Preprocess: dropna
    reviews_df = data[['app','reviews']].dropna().copy()
    # If 'reviews' contains concatenated reviews or counts, you'll need an actual reviews dataset for meaningful sentiment analysis.
    # We'll assume 'reviews' contains text sentences (or short text).
    # Compute polarity and subjectivity
    def get_sentiment(text):
        try:
            blob = TextBlob(str(text))
            return blob.sentiment.polarity, blob.sentiment.subjectivity
        except:
            return np.nan, np.nan

    reviews_df[['polarity','subjectivity']] = reviews_df['reviews'].apply(lambda x: pd.Series(get_sentiment(x)))
    display(reviews_df.head())

    # Average polarity per app (sample)
    avg_sent = reviews_df.groupby('app')['polarity'].mean().reset_index().sort_values('polarity', ascending=False)
    display(avg_sent.head(10))

    # Histogram of polarity
    fig = px.histogram(reviews_df, x='polarity', nbins=40, title='Sentiment Polarity Distribution of Reviews')
    fig.show()
else:
    print("No textual 'reviews' column detected. If you have a separate reviews CSV, load it and rerun sentiment analysis.")

,app,reviews,polarity,subjectivity
0,Photo Editor & Candy Camera & Grid & ScrapBook,159,0.0,0.0
1,Coloring book moana,967,0.0,0.0
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",87510,0.0,0.0
3,Sketch - Draw & Paint,215644,0.0,0.0
4,Pixel Draw - Number Art Coloring Book,967,0.0,0.0


,app,polarity
9658,🔥 Football Wallpapers 4K | Full HD Backgrounds 😍,0.0
0,"""i DT"" Fútbol. Todos Somos Técnicos.",0.0
1,+Download 4 Instagram Twitter,0.0
2,- Free Comics - Comic Apps,0.0
3,.R,0.0
4,/u/app,0.0
5,058.ba,0.0
6,1. FC Köln App,0.0
7,10 Best Foods for You,0.0
8,10 Minutes a Day Times Tables,0.0


In [12]:
# Word Cloud requires wordcloud library — optional install and run if needed.
# !pip install wordcloud --quiet
from wordcloud import WordCloud, STOPWORDS

if 'description' in data.columns:
    text = " ".join(str(txt) for txt in data['description'].dropna().astype(str).tolist())
    stopwords = set(STOPWORDS)
    wc = WordCloud(width=1200, height=600, stopwords=stopwords, background_color='white').generate(text)
    plt.figure(figsize=(12,6))
    plt.imshow(wc, interpolation='bilinear')
    plt.axis('off')
    plt.title('Word Cloud from App Descriptions')
    plt.show()
else:
    print("No 'description' column available for wordcloud generation.")

No 'description' column available for wordcloud generation.


In [14]:
# Build an interactive scatter: installs vs rating colored by category
if {'installs_log','rating','category','app'}.issubset(data.columns):
    sample = data[data['rating'].notna()].sample(min(2000, len(data)), random_state=42)
    fig = px.scatter(sample, x='installs_log', y='rating', color='category', hover_data=['app','installs'],
                     title='Interactive: Log(Installs) vs Rating by Category', height=700)
    fig.update_layout(legend={'itemsizing':'constant'})
    fig.show()


/usr/local/lib/python3.12/dist-packages/plotly/express/_core.py:1992: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [13]:
# Print short insights template — edit after reviewing visuals & results
insights = [
    "1. Category Distribution: List top categories and what share of apps they hold.",
    "2. Ratings: Most categories have median ratings around X — highlight high/low performers.",
    "3. Installs & Popularity: A small portion of apps receive the majority of installs (long-tail effect).",
    "4. Price Trends: Paid apps often belong to niche categories and have varied ratings.",
    "5. Sentiment: Textual review sentiment (if available) helps identify features users like/dislike.",
    "6. Recommendation: For app developers — target under-served categories with good rating potential; consider monetization strategies based on installs vs price patterns."
]

for line in insights:
    print(line)

1. Category Distribution: List top categories and what share of apps they hold.
2. Ratings: Most categories have median ratings around X — highlight high/low performers.
3. Installs & Popularity: A small portion of apps receive the majority of installs (long-tail effect).
4. Price Trends: Paid apps often belong to niche categories and have varied ratings.
5. Sentiment: Textual review sentiment (if available) helps identify features users like/dislike.
6. Recommendation: For app developers — target under-served categories with good rating potential; consider monetization strategies based on installs vs price patterns.


In [16]:
# Save cleaned file for future use
OUT_PATH = "/content/googleplaystore_cleaned.csv"
data.to_csv(OUT_PATH, index=False)
print("Saved cleaned dataset to:", OUT_PATH)


Saved cleaned dataset to: /content/googleplaystore_cleaned.csv
